In [1]:
!pip install accelerate \
    protobuf \
    sentencepiece \
    torch \
    transformers \
    huggingface_hub \
    datasets \
    bert_score \
    sentence-transformers \
    evaluate \
    rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=801548f686b8e858ed8232fe0b8545f65d8eab1cdf57890339141dfd115c479d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fss

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login


model_name = "chandankh/Qwen2.5-0.5B-Instruct-fine-tuned-16bit"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to generate responses
def chat_with_qwen(prompt):
    messages = [
        {"role": "system", "content": "You are Qwen, You are a helpful assistant for students"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=200)
    generated_text = tokenizer.batch_decode(generated_ids[:, model_inputs['input_ids'].size(1):], skip_special_tokens=True)[0]
    print("Response:", generated_text)
    return generated_text


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import evaluate
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util

# Load dataset
dataset = pd.read_csv("New_Test_Dataset.csv")  # Replace with your CSV file path
batch_size = 10

# Initialize evaluation metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
sbert_model = SentenceTransformer('all-mpnet-base-v2')

# Initialize lists to store results
generated_answers = []
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
bert_scores = []
sbert_cosine_scores = []

# Batch processing
for i in range(0, len(dataset), batch_size):
    batch = dataset.iloc[i:i + batch_size]
    for index, row in batch.iterrows():
        question = row['question']
        ground_truth = row['ground_truth']

        # Check for NaN values
        if pd.isna(question) or pd.isna(ground_truth):
            print(f"Skipping NaN value at index {index}")
            continue

        # Generate model answer
        model_answer = chat_with_qwen(question)
        generated_answers.append(model_answer)

        # Compute BLEU score
        bleu = bleu_metric.compute(predictions=[model_answer], references=[ground_truth])
        bleu_scores.append(bleu["bleu"])

        # Compute ROUGE scores
        rouge = rouge_metric.compute(predictions=[model_answer], references=[ground_truth])
        rouge1_scores.append(rouge['rouge1'])
        rouge2_scores.append(rouge['rouge2'])
        rougeL_scores.append(rouge['rougeL'])

        # Compute BERTScore
        P, R, F1 = bert_score([model_answer], [ground_truth], lang="en")
        bert_scores.append(F1.mean().item())

        # Compute SBERT cosine similarity
        embedding1 = sbert_model.encode(model_answer, convert_to_tensor=True)
        embedding2 = sbert_model.encode(ground_truth, convert_to_tensor=True)
        cosine_score = util.pytorch_cos_sim(embedding1, embedding2).item()
        sbert_cosine_scores.append(cosine_score)

# Add results to DataFrame
dataset['model_answer'] = generated_answers
dataset['bleu_score'] = bleu_scores
dataset['rouge1_score'] = rouge1_scores
dataset['rouge2_score'] = rouge2_scores
dataset['rougeL_score'] = rougeL_scores
dataset['bert_score'] = bert_scores
dataset['sbert_cosine_score'] = sbert_cosine_scores

# Save results to CSV
dataset.to_csv("evaluation_results_qwen_fine_tuned_model.csv", index=False)
print("Evaluation completed and results saved to evaluation_results_qwen_fine_tuned_model.csv")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Response: The formal sector loans can be made beneficial to poor farmers and workers in several ways:

1. Microfinance Institutions (MFIs) such as Small and Medium Enterprises (SMEs), microfinance cooperatives or cooperative societies can offer small loan facilities at competitive rates to small and medium-sized farmers.
2. Government schemes like National Rural Employment Guarantee Act (NRGEA), Farmer Loan Support Programme (FLSP), etc., provide direct support to rural households through agricultural credit schemes.
3. Banks can issue small loans to small farmers who have no other means of repayment due to lack of access to finance. These loans are typically short-term (one year) and can be repaid with interest over time.
4. The government can invest in infrastructure projects that benefit farmers and help them develop their farming skills and markets. This could include irrigation systems, roads, storage tanks, and modern farm equipment.
5. Farmers can also receive training programs 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Response: Globalization refers to the process of international trade and economic exchange that has led to the creation of new markets and economies around the world. It involves countries sharing access to resources, technology, skills and markets in order to achieve mutual benefit.
Key points about globalization:
- Economic growth: The process creates jobs and wealth for people across borders.
- Trade liberalization: Countries can freely buy goods from other nations without restrictions.
- Technological advancements: Advances in communication, information and technology have made it easier to move goods and ideas between countries.
- Political integration: International agreements like the World Trade Organization facilitate cooperation among nations.
- Cultural exchanges: People from different cultures interact with each other more often through travel, education and cultural exchange programs.
- Environmental concerns: As industries develop globally, there's an increasing need to p

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Response: Indian markets have been transformed by globalization in several ways:

1. Trade: With the growth of trade between India and other countries, there has been an increase in demand for Indian products.
2. Investment: Foreign investors have poured money into India to invest in industries like IT, pharmaceuticals, and renewable energy.
3. Education: The global education system has brought about significant changes in education systems across India.
4. Infrastructure: With increased investment in infrastructure, India has become a hub for innovation and technology.
5. Consumerism: Indian consumers are increasingly buying goods from abroad due to the convenience of shopping online.

These factors have led to an increase in economic growth and job creation, but it also brings challenges such as income inequality and environmental degradation.


In [5]:
results_df = pd.read_csv("evaluation_results_qwen_fine_tuned_model.csv")
average_scores = {
    'avg_bleu': results_df['bleu_score'].mean(),
    'avg_rouge1': results_df['rouge1_score'].mean(),
    'avg_rouge2': results_df['rouge2_score'].mean(),
    'avg_rougeL': results_df['rougeL_score'].mean(),
    'avg_bert_score': results_df['bert_score'].mean(),
    'avg_sbert_cosine': results_df['sbert_cosine_score'].mean()
}

# Display average scores
print("\nAverage Scores:")
for metric, score in average_scores.items():
    print(f"{metric}: {score:.4f}")



Average Scores:
avg_bleu: 0.0217
avg_rouge1: 0.2473
avg_rouge2: 0.0651
avg_rougeL: 0.1645
avg_bert_score: 0.8433
avg_sbert_cosine: 0.7630
